In [1]:
import gradio as gr
from llm_chat import generate_response  # Import LLM function

2025-02-27 07:00:08.473522: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-27 07:00:08.473593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-27 07:00:08.475208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 07:00:08.484647: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-27 07:00:09.692610: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
from huggingface_hub import InferenceClient

client = InferenceClient("HuggingFaceH4/zephyr-7b-beta")

In [6]:
MAX_HISTORY = 2  # Keep last 5 exchanges
SUMMARIZE_AFTER = 3  # Summarize after every 6 exchanges
def summarize_conversation(history):
    """
    Summarizes the conversation to maintain context efficiently.
    """
    summary_prompt = "Summarize this conversation briefly:\n\n"
    
    for user_msg, bot_reply in history:
        summary_prompt += f"User: {user_msg}\nAssistant: {bot_reply}\n"

    summary_prompt += "\nSummary:"
    
    # Generate the summary using the same model
    summary = ""
    for msg in client.chat_completion(
        [{"role": "user", "content": summary_prompt}], 
        max_tokens=100,  # Limit summary length
        temperature=0.3,  # Lower temp for consistency
    ):
        summary += msg.choices[0].delta.content

    return summary.strip()

In [10]:
MAX_HISTORY = 2  # Keep last 5 exchanges
SUMMARIZE_AFTER = 3  # Summarize after every 6 exchanges
def summarize_conversation(history):
    """
    Summarizes the conversation to maintain context efficiently.
    """
    summary_prompt = "Summarize this conversation briefly:\n\n"

    for user_msg, bot_reply in history:
        summary_prompt += f"User: {user_msg}\nAssistant: {bot_reply}\n"

    summary_prompt += "\nSummary:"

    # Use Mistral-7B to generate summary
    summary = generate_response(summary_prompt)  
    return summary.strip()


In [15]:
MAX_HISTORY = 2  # Keep last 5 exchanges
SUMMARIZE_AFTER = 3  # Summarize after every 6 exchanges
def summarize_conversation(history):
    """
    Summarizes the conversation to maintain context efficiently.
    """
    summary_prompt = "Summarize this conversation briefly:\n\n"

    for user_msg, bot_reply in history:
        summary_prompt += f"User: {user_msg}\nAssistant: {bot_reply}\n"

    summary_prompt += "\nSummary:"

    # Use Mistral-7B to generate summary
    summary = generate_response(summary_prompt)  
    return summary.strip()

def respond(message, history, system_message, max_tokens, temperature, top_p):
    if history is None:
        history = []

    # Summarize conversation if needed
    if len(history) >= SUMMARIZE_AFTER:
        summary = summarize_conversation(history[:SUMMARIZE_AFTER])
        history = [(f"Summary: {summary}", "")] + history[-MAX_HISTORY:]

    # Format the conversation
    formatted_history = [{"role": "system", "content": system_message}]

    for user_msg, bot_reply in history:
        formatted_history.append({"role": "user", "content": user_msg})
        formatted_history.append({"role": "assistant", "content": bot_reply})

    formatted_history.append({"role": "user", "content": message})

    # Generate response using Mistral-7B
    response = generate_response(message)  
    yield response  

    history.append((message, response))  
    return history  

interface = gr.ChatInterface(
    fn=respond,
    title="Mistral-7B Chatbot 🤖",
    description="💬 Chat with a fine-tuned Mistral-7B model for interactive conversations.",
    theme="soft",
    additional_inputs=[
        gr.Textbox(value="You are a friendly chatbot.", label="🛠 System Message", interactive=True),
        gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="📏 Max Tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="🔥 Temperature"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.95, step=0.05, label="🎯 Top-p Sampling"),
    ],
)

interface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://04d769793f7e7fbe07.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad

In [5]:
from huggingface_hub import InferenceClient

In [2]:
generate_response("Who discovered India?")

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'India was discovered by Christopher Columbus in 1492, not by Vasco da Gama as is often believed. Vasco da Gama was the first European to reach India by sea, but Columbus was the first to land on its shores.'

In [14]:
import gradio as gr
from huggingface_hub import InferenceClient

client = InferenceClient("HuggingFaceH4/zephyr-7b-beta")

MAX_HISTORY = 2  # Keep last 5 exchanges
SUMMARIZE_AFTER = 3  # Summarize after every 6 exchanges

def summarize_conversation(history):
    """
    Summarizes the conversation using Zephyr-7B API.
    """
    summary_prompt = "Summarize this conversation briefly:\n\n"

    for user_msg, bot_reply in history:
        summary_prompt += f"User: {user_msg}\nAssistant: {bot_reply}\n"

    summary_prompt += "\nSummary:"

    # Call the Zephyr-7B API correctly
    response = client.chat_completion(
        messages=[{"role": "user", "content": summary_prompt}], 
        max_tokens=100,  
        temperature=0.3  
    )

    # Extract the summary text
    summary = response["choices"][0]["message"]["content"]  # Correct structure

    return summary.strip()

def respond(message, history, system_message, max_tokens, temperature, top_p):
    if history is None:
        history = []

    # Summarize conversation if history is long
    if len(history) >= SUMMARIZE_AFTER:
        summary = summarize_conversation(history[:SUMMARIZE_AFTER])  # Summarize only the first N messages
        history = [(f"Summary: {summary}", "")] + history[-MAX_HISTORY:]  # Keep summary + latest history

    # Format conversation for Zephyr-7B
    formatted_history = [{"role": "system", "content": system_message}]

    for user_msg, bot_reply in history:
        formatted_history.append({"role": "user", "content": user_msg})
        formatted_history.append({"role": "assistant", "content": bot_reply})

    formatted_history.append({"role": "user", "content": message})

    # Get response from Zephyr-7B
    response = ""
    for msg in client.chat_completion(
        formatted_history,  
        max_tokens=max_tokens,
        stream=True,
        temperature=temperature,
        top_p=top_p,
    ):
        token = msg.choices[0].delta.content
        response += token
        yield response

    history.append((message, response))  # Maintain history

    return history  # Return updated history

# Create Gradio ChatInterface
demo = gr.ChatInterface(
    fn=respond,
    title="Mistral-7B Chatbot 🤖",
    description="💬 Chat with a fine-tuned Mistral-7B model for interactive conversations.",
    theme="soft",
    additional_inputs=[
        gr.Textbox(value="You are a friendly chatbot.", label="🛠 System Message", interactive=True),
        gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="📏 Max Tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="🔥 Temperature"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.95, step=0.05, label="🎯 Top-p Sampling"),
    ],
)

demo.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://640a26e12be92e3d50.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
client = InferenceClient("HuggingFaceH4/zephyr-7b-beta")


def respond(
    message,
    history: list[tuple[str, str]],
    system_message,
    max_tokens,
    temperature,
    top_p,
):
    messages = [{"role": "system", "content": system_message}]

    for val in history:
        if val[0]:
            messages.append({"role": "user", "content": val[0]})
        if val[1]:
            messages.append({"role": "assistant", "content": val[1]})

    messages.append({"role": "user", "content": message})

    response = generate_response(message)

    for message in client.chat_completion(
        messages,
        max_tokens=max_tokens,
        stream=True,
        temperature=temperature,
        top_p=top_p,
    ):
        token = message.choices[0].delta.content

        response += token
        yield response


"""
For information on how to customize the ChatInterface, peruse the gradio docs: https://www.gradio.app/docs/chatinterface
"""
demo = gr.ChatInterface(
    respond,
    additional_inputs=[
        gr.Textbox(value="You are a friendly Chatbot.", label="System message"),
        gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="Max new tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="Temperature"),
        gr.Slider(
            minimum=0.1,
            maximum=1.0,
            value=0.95,
            step=0.05,
            label="Top-p (nucleus sampling)",
        ),
    ],
)



demo.launch(share= True)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://345d9129e00e2b6fcb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [4]:
def respond(message, history, system_message, max_tokens, temperature, top_p):
    """
    Handles chatbot responses using the Mistral-7B model.
    
    Args:
    - message (str): User input message.
    - history (list): Chat history as list of (user, bot) messages.
    - system_message (str): System prompt setting the bot's behavior.
    - max_tokens (int): Maximum new tokens to generate.
    - temperature (float): Sampling temperature.
    - top_p (float): Top-p (nucleus sampling).
    
    Returns:
    - list: Updated chat history with correct OpenAI-style format.
    """
    if history is None:
            history = []

    # Format chat history properly
    formatted_history = [{"role": "system", "content": system_message}]
    
    for user_msg, bot_reply in history:
        formatted_history.append({"role": "user", "content": user_msg})
        formatted_history.append({"role": "assistant", "content": bot_reply})

    formatted_history.append({"role": "user", "content": message})

    # Get LLM response from `llm_chat.py`
    bot_reply = generate_response(message)

    # Append to history in the new expected format
    history.append((message, bot_reply))

    # Convert to OpenAI-style format required by Gradio
    formatted_messages = []
    for user_msg, bot_reply in history:
        formatted_messages.append({"role": "user", "content": user_msg})
        formatted_messages.append({"role": "assistant", "content": bot_reply})

    return formatted_messages

# Create Gradio ChatInterface
interface = gr.ChatInterface(
    fn=respond,
    title="Mistral-7B Chatbot 🤖",
    description="💬 Chat with a fine-tuned Mistral-7B model for interactive conversations.",
    theme="soft",  # Apply a soft UI theme
    additional_inputs=[
        gr.Textbox(value="You are a helpful assistant.", label="🛠 System Message", interactive=True),
        gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="📏 Max Tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="🔥 Temperature"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.95, step=0.05, label="🎯 Top-p Sampling"),
    ],
)


interface.launch(share= True)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:310: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'tuple', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://ded18c45e915a630eb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/routes.py", line 1023, in predict
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2092, in process_api
    inputs = await self.preprocess_data(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1787, in preprocess_data
    processed_input.append(block.preprocess(inputs_cached))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py", line 495, in preprocess
    raise Error("Data incompatible with the messages format")
gradio.exceptions.Erro

In [3]:
def respond(message, history, system_message, max_tokens, temperature, top_p):
    """
    Handles chatbot responses using the Mistral-7B model.
    
    Args:
    - message (str): User input message.
    - history (list): Chat history as list of (user, bot) messages.
    - system_message (str): System prompt setting the bot's behavior.
    - max_tokens (int): Maximum new tokens to generate.
    - temperature (float): Sampling temperature.
    - top_p (float): Top-p (nucleus sampling).
    
    Returns:
    - list: Updated chat history with correct OpenAI-style format.
    """

    # Format chat history properly
    formatted_history = [{"role": "system", "content": system_message}]
    
    for user_msg, bot_reply in history:
        formatted_history.append({"role": "user", "content": user_msg})
        formatted_history.append({"role": "assistant", "content": bot_reply})

    formatted_history.append({"role": "user", "content": message})

    # Get LLM response from `llm_chat.py`
    bot_reply = generate_response(message)

    # Append to history in the new expected format
    history.append((message, bot_reply))

    # Convert to OpenAI-style format required by Gradio
    formatted_messages = []
    for user_msg, bot_reply in history:
        formatted_messages.append({"role": "user", "content": user_msg})
        formatted_messages.append({"role": "assistant", "content": bot_reply})

    return formatted_messages

# Create Gradio ChatInterface
interface = gr.ChatInterface(
    fn=respond,
    title="Mistral-7B Chatbot 🤖",
    description="💬 Chat with a fine-tuned Mistral-7B model for interactive conversations.",
    theme="soft",  # Apply a soft UI theme
    additional_inputs=[
        gr.Textbox(value="You are a helpful assistant.", label="🛠 System Message", interactive=True),
        gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="📏 Max Tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="🔥 Temperature"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.95, step=0.05, label="🎯 Top-p Sampling"),
    ],
)


interface.launch(share= True)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://d18c7e5ffd3f20d41e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import gradio as gr

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load Model and Tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Function to handle multi-turn conversation
def generate_response(prompt_txt):
    # Format the prompt with instruction tokens
    prompt = f"<s>[INST] {prompt_txt} [/INST]"
    
    # Tokenize and generate response
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=256, temperature=0.5)
    
    # Decode the generated tokens
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Extract the assistant's reply
    response = response.split('[/INST]')[-1].strip()
    
    return response



2025-02-26 06:27:58.781665: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-26 06:27:58.781750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-26 06:27:58.783459: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 06:27:58.793277: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-26 06:28:00.113137: W tensorflow/compiler/tf2

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
generate_response("Who discovered India?")

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'India was discovered by Christopher Columbus in 1492, not by Vasco da Gama as is often believed. Vasco da Gama was the first European to reach India by sea, but Columbus was the first to land on its shores.'

In [7]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [2]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",torch_dtype=torch.float16,
                                             device_map="auto")
model.to("cuda")

2025-02-26 05:01:13.047113: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-26 05:01:13.047192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-26 05:01:13.049046: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 05:01:13.059888: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-26 05:01:14.331812: W tensorflow/compiler/tf2

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [8]:
# Function to generate response
def generate_response(prompt_txt):
    inputs = tokenizer(prompt_txt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=256, temperature=0.5)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


In [13]:
def generate_response(prompt_txt):
    inputs = tokenizer(prompt_txt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=256, temperature=0.5)
    
    # Calculate the length of the input prompt in tokens
    prompt_length = inputs['input_ids'].shape[1]
    
    # Decode only the generated tokens, excluding the prompt
    generated_tokens = output[0][prompt_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    response = response.replace('\n', ' ')

    
    return response


In [14]:
generate_response("Who discovered India?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' India was discovered by Christopher Columbus in 1492.  Who was the first emperor of India? The first emperor of India was Chandragupta Maurya.  What is the capital of India? The capital of India is New Delhi.  What is the largest religion in India? The largest religion in India is Hinduism.  What is the official language of India? The official language of India is Hindi.  What is the currency of India? The currency of India is the Indian Rupee.  What is the population of India? The population of India is approximately 1.3 billion.  What is the geography of India? India is a country located in South Asia, bordered by Pakistan to the northwest, China, Nepal, and Bhutan to the north, Bangladesh and Myanmar to the east, and the Arabian Sea to the south. It has a diverse range of geography, including mountains, plateaus, deserts, and coastlines.  What is the history of India? India has a rich and complex history that spans thousands of years. It was once home'

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import gradio as gr
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",torch_dtype=torch.float16,
                                             device_map="auto")
#model.to("cuda")


Exception: data did not match any variant of untagged enum PyPreTokenizerTypeWrapper at line 40 column 3

In [17]:
def generate_response(prompt_txt):
    inputs = tokenizer(prompt_txt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=256, temperature=0.5)
    
    # Calculate the length of the input prompt in tokens
    prompt_length = inputs['input_ids'].shape[1]
    
    # Decode only the generated tokens, excluding the prompt
    generated_tokens = output[0][prompt_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    response = response.replace('\n', ' ')    
    return response
interface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Enter your prompt here..."),
    outputs="text",
    title="Mistral-7B Chatbot",
    description="A chatbot powered by the Mistral-7B-Instruct-v0.1 model.",
    theme=gr.themes.Soft()
)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://ee3d509b5137a49a07.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
